In [ ]:
# train.csv 파일 불러온 후 real은 real끼리, fake는 fake끼리 묶음
# real인 오디오와 fake인 오디오를 각각 하나로 합침
# train.csv 의 label을 각각 real, fake 열로 만들어 1과 0으로 나타냄
# real과 fake가 합쳐진 오디오는 real도 1, fake도 1로 설정

In [18]:
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('./train.csv')
reals_id = []
reals_path = []
reals_label = []

fakes_id = []
fakes_path = []
fakes_label = []

for _, row in tqdm(df.iterrows()):
    if row['label'] == 'real':
        reals_id.append(row['id'])
        reals_path.append(row['path'])
        reals_label.append(row['label'])
    else:
        fakes_id.append(row['id'])
        fakes_path.append(row['path'])
        fakes_label.append(row['label'])

reals = pd.DataFrame({'id': reals_id, 'path': reals_path, 'label': reals_label})
fakes = pd.DataFrame({'id': fakes_id, 'path': fakes_path, 'label': fakes_label})
reals.to_csv('train_reals.csv', index=False)
fakes.to_csv('train_fakes.csv', index=False)

55438it [00:01, 30303.83it/s]


In [28]:
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡreal 과 fake 합치는 코드ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#
reals = pd.read_csv('./train_reals.csv')
fakes = pd.read_csv('./train_fakes.csv')

for i in tqdm(range(len(reals))):
    # 오디오 파일 경로
    audio_path1 = reals['path'].iloc[i]
    audio_path2 = fakes['path'].iloc[i]

    y1, sr1 = librosa.load(audio_path1, sr=None)
    y2, sr2 = librosa.load(audio_path2, sr=None)

    # sr이 같아야함
    if sr1 != sr2:
        raise ValueError('Sample rates of the two audio files are different.')

    # 짧은 쪽의 오디오길이로 맞춤
    min_length = min(len(y1), len(y2))
    y1 = y1[:min_length]
    y2 = y2[:min_length]

    y_combined = y1 + y2
    output_path = f'./mixed/mixed_{i}.ogg' # (1)
    sf.write(output_path, y_combined, sr1)

#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡreal 과 fake 합치는 코드ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ#

100%|██████████| 27620/27620 [12:44<00:00, 36.14it/s]


In [3]:
import os

directory = './mixed/' # (2)

id = []
path = []

# 디렉토리 내의 모든 파일들에 대해 반복
for filename in os.listdir(directory):
    if filename.endswith(".ogg"):
        name = filename.split('.')
        id.append(name[0])
        path.append(f'./train/{filename}') 

mix = pd.DataFrame({'id': id, 'path': path, 'real': 1, 'fake': 1})

In [33]:

df = pd.read_csv('./train.csv')

#예) label이 real일 경우, real열에 1, fake열에 0 넣는 부분
df['real'] = df['label'].apply(lambda x: 1 if x == 'real' else 0)
df['fake'] = df['label'].apply(lambda x: 1 if x == 'fake' else 0)

# 기존의 label 열 삭제
df.drop(columns=['label'], inplace=True)

conc = pd.concat([df, mix])
conc.to_csv('train_mixed.csv', index=False)